oversampling method 

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix


In [2]:
df=sns.load_dataset("iris")

In [3]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [4]:
(df["species"]).unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [5]:
# Map species to numeric
class_map = {"setosa": 0, "versicolor": 1, "virginica": 2}
class_unmap = {0:"setosa",1: "versicolor", 2:"virginica"}
df["species"] = df["species"].replace(class_map)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_25024\918621057.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["species"] = df["species"].replace(class_map)


# model

In [6]:
def linear_regression(X, y, alpha=0.1, epochs=1000):
    n_samples, n_features = X.shape
    w = np.zeros((n_features, 1))       # column vector
    y = y.reshape(-1, 1)                # ensure column vector
    loss_history = []

    for _ in range(epochs):
        y_pred = X @ w                  # (n_samples,1)
        error = y_pred - y              # (n_samples,1)
        gradient = (1 / n_samples) * (X.T @ error)  # (n_features,1)
        w -= alpha * gradient           # update
        loss = (1 / (2 * n_samples)) * np.sum(error ** 2)
        loss_history.append(loss)

    return w, loss_history

In [7]:
# Prediction
def predict(X, w):
    return X.dot(w)

# Convert regression outputs → class labels
def classify(preds):
    return np.round(preds).astype(int).clip(0, 2)

In [8]:
# Evaluation metrics
def evaluate(y_true, y_pred, name="Model"):
    accuracy = np.mean(y_true == y_pred)
    precision_list, recall_list, f1_list = [], [], []

    for c in np.unique(y_true):
        tp = np.sum((y_pred == c) & (y_true == c))
        fp = np.sum((y_pred == c) & (y_true != c))
        fn = np.sum((y_pred != c) & (y_true == c))

        precision = tp / (tp + fp + 1e-9)
        recall = tp / (tp + fn + 1e-9)
        f1 = 2 * precision * recall / (precision + recall + 1e-9)

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    print(f"\n{name} Results:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {np.mean(precision_list):.2f}")
    print(f"Recall: {np.mean(recall_list):.2f}")
    print(f"F1 Score: {np.mean(f1_list):.2f}")


In [9]:
def evaluate_model(X, y, w, name="Model"):
    predictions = [sum(w[j] * x[j] for j in range(len(x))) + w[-1] for x in X]
    avg_error = np.mean(np.abs(np.array(predictions) - y))
    print(f"{name} Average Absolute Error: {avg_error:.2f}")

# split

In [10]:
def one_hot_encode(labels):
    classes = sorted(set(labels))
    class_to_idx = {c: i for i, c in enumerate(classes)}
    Y = np.zeros((len(labels), len(classes)))
    for i, label in enumerate(labels):
        Y[i, class_to_idx[label]] = 1
    return Y, class_to_idx

In [11]:
def split(X, y, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1."

    np.random.seed(42)
    indices = np.arange(len(X))
    np.random.shuffle(indices)

    # Ensure NumPy arrays
    X = np.array(X)
    y = np.array(y)

    X = X[indices]
    y = y[indices]

    n_train = int(train_ratio * len(X))
    n_val = int(val_ratio * len(X))

    X_train, y_train = X[:n_train], y[:n_train]
    X_val, y_val = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
    X_test, y_test = X[n_train+n_val:], y[n_train+n_val:]

    return X_train, y_train, X_val, y_val, X_test, y_test


# over sample

In [ ]:
def oversample(X, y, target_class):
    X = np.array(X) #feature data
    y = np.array(y)#class labels

    X_new, y_new = [], []
    unique, counts = np.unique(y, return_counts=True) #unique classes and their counts
    max_count = max(counts) #sabse zyada kon se class ke sample hai

    for c in unique:
        X_c = X[y == c]
        y_c = y[y == c]

        if c == target_class:
            while len(X_c) < max_count:
                idx = np.random.randint(0, len(X_c))  
                X_c = np.vstack([X_c, X_c[idx]]) #add vertically feature data
                y_c = np.append(y_c, c)

        X_new.append(X_c)
        y_new.append(y_c)

    return np.vstack(X_new), np.concatenate(y_new)


# implementation

In [13]:
X=df.drop(columns="species")
y=df["species"]

In [14]:
cols=X.columns
cols

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [15]:
def min_max(df):
    return (df-df.min())/(df.max()-df.min())

In [16]:
for c in cols:
    X[c]=min_max(X[c])

In [17]:
for minority_class in [0, 1, 2]:
    print(f"\n=== Minority Class: {minority_class} ===")

    mask = y != minority_class
    keep_idx = np.where(y == minority_class)[0]
    keep_idx = np.random.choice(keep_idx, size=len(keep_idx)//2, replace=False)
    mask[keep_idx] = True

    X_reduced, y_reduced = X[mask], y[mask]

    X_bal, y_bal = oversample(X_reduced, y_reduced, minority_class)

    X_train, y_train, X_val, y_val, X_test, y_test = split(X_bal, y_bal)

    w,_ = linear_regression(X_train, y_train, alpha=0.01, epochs=2000)

    y_pred = classify(predict(X_test, w))
    evaluate(y_test, y_pred, name=f"Classifier (minority={class_unmap[minority_class]})")


=== Minority Class: 0 ===

Classifier (minority=setosa) Results:
Accuracy: 0.36
Precision: 0.33
Recall: 0.33
F1 Score: 0.33

=== Minority Class: 1 ===

Classifier (minority=versicolor) Results:
Accuracy: 0.35
Precision: 0.33
Recall: 0.33
F1 Score: 0.33

=== Minority Class: 2 ===

Classifier (minority=virginica) Results:
Accuracy: 0.37
Precision: 0.33
Recall: 0.33
F1 Score: 0.33
